# Prompt Engineering Tutorial with LangChain

This comprehensive tutorial covers various prompt engineering techniques using LangChain and ChatOpenAI.

## Table of Contents
1. Introduction & Setup
2. Basic Prompt Techniques
3. Template Variables & Formatting
4. Prompting Techniques
   - Zero-shot Prompting
   - One-shot Prompting
   - Few-shot Prompting
   - Chain-of-Thought (CoT) Prompting
   - Tree-of-Thought (ToT) Prompting
   - Self-Consistency Prompting
   - ReAct Prompting
   - Role-based Prompting
5. Output Parsers
6. LCEL Chains
7. Best Practices
8. Real-world Examples

## 1. Introduction & Setup

**What is Prompt Engineering?**

Prompt engineering is the practice of designing and optimizing prompts to get the best possible responses from Large Language Models (LLMs). It involves:
- Crafting clear, specific instructions
- Providing appropriate context and examples
- Structuring prompts for desired output formats
- Iterating and refining based on results

In [ ]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Load environment variables
load_dotenv()

# Initialize ChatOpenAI model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

print("✅ Setup complete!")

## 2. Basic Prompt Techniques

### Simple Prompts with ChatPromptTemplate

In [ ]:
# Simple prompt using ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "What is the capital of France?")
])

# Create chain and invoke
chain = prompt | model | StrOutputParser()
response = chain.invoke({})
print(response)

### Message Types: System, Human, and AI Messages

In [ ]:
# Using different message types
messages = [
    SystemMessage(content="You are a Python programming expert."),
    HumanMessage(content="What is a list comprehension?"),
]

response = model.invoke(messages)
print(response.content)

## 3. Template Variables & Formatting

Templates allow you to create reusable prompts with dynamic variables.

In [ ]:
# Template with variables
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
    ("human", "{text}")
])

chain = prompt | model | StrOutputParser()

response = chain.invoke({
    "input_language": "English",
    "output_language": "French",
    "text": "I love programming"
})

print(response)

## 4. Prompting Techniques

### Zero-shot Prompting

**Definition:** Direct instruction without any examples.

**Importance:** Tests the model's inherent knowledge and capabilities.

**Use Cases:** Simple tasks, general knowledge queries, when examples aren't available.

In [ ]:
# Zero-shot prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analysis expert."),
    ("human", "Classify the sentiment of this text as positive, negative, or neutral: {text}")
])

chain = prompt | model | StrOutputParser()

# Test with different texts
texts = [
    "I absolutely love this product! It's amazing!",
    "This is the worst experience I've ever had.",
    "The weather is cloudy today."
]

print("Zero-shot Sentiment Analysis:\n")
for text in texts:
    result = chain.invoke({"text": text})
    print(f"Text: {text}")
    print(f"Sentiment: {result}\n")

### One-shot Prompting

**Definition:** Providing a single example to guide the model.

**Importance:** Demonstrates desired format/style with minimal context.

**Use Cases:** Format specification, style guidance, simple pattern recognition.

In [ ]:
# One-shot prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an email classifier. Classify emails as 'spam' or 'not spam'."),
    ("human", "Email: Get rich quick! Click here now!"),
    ("ai", "Classification: spam"),
    ("human", "Email: {email}")
])

chain = prompt | model | StrOutputParser()

email = "Meeting scheduled for tomorrow at 3 PM in conference room B."
result = chain.invoke({"email": email})

print(f"Email: {email}")
print(f"Result: {result}")

### Few-shot Prompting

**Definition:** Providing 2-5 examples to establish patterns.

**Importance:** Significantly improves accuracy for specific tasks without fine-tuning.

**Use Cases:** Classification, data extraction, consistent formatting, domain-specific tasks.

In [ ]:
# Few-shot prompting example
examples = [
    {"input": "The movie was fantastic! I loved every minute.", "output": "positive"},
    {"input": "Terrible service, will never come back.", "output": "negative"},
    {"input": "The product works as expected.", "output": "neutral"},
]

# Create few-shot prompt template
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment classifier. Classify text as positive, negative, or neutral."),
    few_shot_prompt,
    ("human", "{input}"),
])

chain = final_prompt | model | StrOutputParser()

# Test with new input
new_text = "The food was okay, nothing special."
result = chain.invoke({"input": new_text})

print(f"Text: {new_text}")
print(f"Sentiment: {result}")

### Chain-of-Thought (CoT) Prompting

**Definition:** Encouraging step-by-step reasoning.

**Importance:** Dramatically improves performance on complex reasoning tasks.

**Use Cases:** Math problems, logical reasoning, multi-step analysis.

In [ ]:
# Chain-of-Thought prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a math tutor. Solve problems step by step."),
    ("human", """Solve this problem step by step:
    
A store has 45 apples. They sell 18 apples in the morning and 12 apples in the afternoon. 
Then they receive a delivery of 30 more apples. How many apples does the store have now?

Let's think step by step:""")
])

chain = prompt | model | StrOutputParser()
result = chain.invoke({})

print(result)

### Tree-of-Thought (ToT) Prompting

**Definition:** Exploring multiple reasoning paths simultaneously.

**Importance:** Enables exploration of different solution strategies.

**Use Cases:** Complex problem-solving, creative tasks, strategic planning.

In [ ]:
# Tree-of-Thought prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a strategic problem solver."),
    ("human", """Problem: How can a small coffee shop increase revenue?

Generate 3 different solution approaches:
1. Approach A: Focus on...
2. Approach B: Focus on...
3. Approach C: Focus on...

Then evaluate each approach and recommend the best one.""")
])

chain = prompt | model | StrOutputParser()
result = chain.invoke({})

print(result)

### Self-Consistency Prompting

**Definition:** Generating multiple reasoning paths and selecting the most consistent answer.

**Importance:** Improves reliability and reduces errors.

**Use Cases:** Critical decisions, fact verification, complex reasoning.

In [ ]:
# Self-consistency prompting example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a logical reasoning expert."),
    ("human", """Problem: If all roses are flowers, and some flowers fade quickly, 
can we conclude that some roses fade quickly?

Solve this problem using 3 different reasoning approaches, then determine the most consistent answer.""")
])

chain = prompt | model | StrOutputParser()
result = chain.invoke({})

print(result)

### ReAct Prompting (Reasoning + Acting)

**Definition:** Combining reasoning with tool/action execution.

**Importance:** Enables agents to interact with external tools and APIs.

**Use Cases:** Web search, database queries, API calls, multi-step workflows.

In [ ]:
# ReAct prompting example (simulated)
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI assistant that uses the ReAct (Reasoning + Acting) framework.
For each task, follow this pattern:
1. Thought: Reason about what to do
2. Action: Specify what action to take
3. Observation: What you would observe from that action
4. Repeat until you can provide a final answer"""),
    ("human", "Task: Find the current population of Tokyo and compare it to New York City.")
])

chain = prompt | model | StrOutputParser()
result = chain.invoke({})

print(result)

### Role-based Prompting

**Definition:** Assigning specific personas or expertise to the model.

**Importance:** Tailors responses to specific domains and audiences.

**Use Cases:** Expert advice, creative writing, technical documentation.

In [ ]:
# Role-based prompting example
roles = [
    "You are a senior Python developer with 10 years of experience.",
    "You are a creative writer specializing in science fiction.",
    "You are a financial advisor helping clients with retirement planning."
]

questions = [
    "What are the best practices for error handling?",
    "Write a short story opening about time travel.",
    "How should I diversify my investment portfolio?"
]

for role, question in zip(roles, questions):
    prompt = ChatPromptTemplate.from_messages([
        ("system", role),
        ("human", question)
    ])
    
    chain = prompt | model | StrOutputParser()
    result = chain.invoke({})
    
    print(f"Role: {role}")
    print(f"Question: {question}")
    print(f"Response: {result[:200]}...\n")
    print("-" * 80 + "\n")

## 5. Output Parsers

Output parsers help structure the model's responses into specific formats.

### String Output Parser

In [ ]:
# String output parser (already used above)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "Tell me a fun fact about {topic}")
])

chain = prompt | model | StrOutputParser()
result = chain.invoke({"topic": "space"})

print(f"Type: {type(result)}")
print(f"Result: {result}")

### Pydantic Output Parser

In [ ]:
# Define Pydantic model
class Person(BaseModel):
    name: str = Field(description="Person's name")
    age: int = Field(description="Person's age")
    occupation: str = Field(description="Person's occupation")
    hobbies: List[str] = Field(description="List of hobbies")

# Create parser
parser = PydanticOutputParser(pydantic_object=Person)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract person information from the text.\n{format_instructions}"),
    ("human", "{text}")
])

chain = prompt | model | parser

text = """John Smith is a 35-year-old software engineer. 
He enjoys hiking, photography, and playing guitar in his free time."""

result = chain.invoke({
    "text": text,
    "format_instructions": parser.get_format_instructions()
})

print(f"Type: {type(result)}")
print(f"Name: {result.name}")
print(f"Age: {result.age}")
print(f"Occupation: {result.occupation}")
print(f"Hobbies: {result.hobbies}")

### JSON Output Parser

In [ ]:
# JSON output parser
json_parser = JsonOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", """Extract product information and return as JSON with these fields:
- product_name
- price
- category
- in_stock (boolean)"""),
    ("human", "{text}")
])

chain = prompt | model | json_parser

text = "The iPhone 15 Pro costs $999 and is available in the Electronics category. Currently in stock."
result = chain.invoke({"text": text})

print(f"Type: {type(result)}")
print(f"Result: {result}")

## 6. LCEL Chains

LangChain Expression Language (LCEL) allows you to chain components using the pipe operator `|`.

In [ ]:
# Multi-step chain example
# Step 1: Generate a topic
topic_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a creative topic generator."),
    ("human", "Generate a random interesting topic about {subject}")
])

# Step 2: Write about the topic
writing_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a skilled writer."),
    ("human", "Write a brief paragraph about: {topic}")
])

# Create chains
topic_chain = topic_prompt | model | StrOutputParser()
writing_chain = writing_prompt | model | StrOutputParser()

# Execute
subject = "artificial intelligence"
topic = topic_chain.invoke({"subject": subject})
paragraph = writing_chain.invoke({"topic": topic})

print(f"Generated Topic: {topic}\n")
print(f"Paragraph: {paragraph}")

## 7. Best Practices

### Clear and Specific Instructions

In [ ]:
# Bad prompt (vague)
bad_prompt = ChatPromptTemplate.from_messages([
    ("human", "Tell me about dogs")
])

# Good prompt (specific)
good_prompt = ChatPromptTemplate.from_messages([
    ("human", """Provide 3 key characteristics of Golden Retrievers as family pets, 
focusing on temperament, exercise needs, and grooming requirements. 
Keep each point to 2 sentences.""")
])

print("Bad Prompt Result:")
result1 = (bad_prompt | model | StrOutputParser()).invoke({})
print(result1[:200] + "...\n")

print("\nGood Prompt Result:")
result2 = (good_prompt | model | StrOutputParser()).invoke({})
print(result2)

### Temperature and Parameter Tuning

In [ ]:
# Compare different temperatures
prompt = ChatPromptTemplate.from_messages([
    ("human", "Write a creative opening line for a mystery novel.")
])

temperatures = [0.0, 0.7, 1.5]

for temp in temperatures:
    model_temp = ChatOpenAI(model="gpt-4o-mini", temperature=temp)
    chain = prompt | model_temp | StrOutputParser()
    result = chain.invoke({})
    print(f"Temperature {temp}: {result}\n")

## 8. Real-world Examples

### Content Summarization

In [ ]:
# Summarization example
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert at summarizing text concisely."),
    ("human", "Summarize the following text in 2-3 sentences:\n\n{text}")
])

chain = prompt | model | StrOutputParser()

long_text = """Artificial Intelligence (AI) has transformed numerous industries over the past decade. 
From healthcare to finance, AI systems are being deployed to automate tasks, provide insights, 
and improve decision-making. Machine learning, a subset of AI, enables computers to learn from 
data without explicit programming. Deep learning, which uses neural networks with multiple layers, 
has achieved remarkable results in image recognition, natural language processing, and game playing. 
However, AI also raises important ethical questions about privacy, bias, and job displacement."""

summary = chain.invoke({"text": long_text})
print(f"Summary: {summary}")

### Data Extraction

In [ ]:
# Data extraction with structured output
class Event(BaseModel):
    event_name: str = Field(description="Name of the event")
    date: str = Field(description="Date of the event")
    location: str = Field(description="Location of the event")
    attendees: int = Field(description="Number of attendees")

parser = PydanticOutputParser(pydantic_object=Event)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract event information from the text.\n{format_instructions}"),
    ("human", "{text}")
])

chain = prompt | model | parser

text = """The Annual Tech Conference will be held on March 15, 2024, at the 
San Francisco Convention Center. We're expecting around 5000 attendees."""

result = chain.invoke({
    "text": text,
    "format_instructions": parser.get_format_instructions()
})

print(f"Event: {result.event_name}")
print(f"Date: {result.date}")
print(f"Location: {result.location}")
print(f"Attendees: {result.attendees}")

### Question Answering System

In [ ]:
# Q&A with context
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant. Answer questions based only on the provided context. 
If the answer is not in the context, say "I don't have enough information to answer that."""),
    ("human", "Context: {context}\n\nQuestion: {question}")
])

chain = prompt | model | StrOutputParser()

context = """Python is a high-level, interpreted programming language created by Guido van Rossum 
and first released in 1991. It emphasizes code readability with significant whitespace. 
Python supports multiple programming paradigms including procedural, object-oriented, and functional programming."""

questions = [
    "Who created Python?",
    "When was Python first released?",
    "What is Python's latest version?"  # Not in context
]

for question in questions:
    answer = chain.invoke({"context": context, "question": question})
    print(f"Q: {question}")
    print(f"A: {answer}\n")

## Summary

In this tutorial, we covered:

1. **Basic Prompts**: ChatPromptTemplate and message types
2. **Template Variables**: Dynamic prompt creation
3. **Prompting Techniques**:
   - Zero-shot: Direct instructions
   - One-shot: Single example
   - Few-shot: Multiple examples
   - Chain-of-Thought: Step-by-step reasoning
   - Tree-of-Thought: Multiple reasoning paths
   - Self-Consistency: Multiple solutions
   - ReAct: Reasoning + Acting
   - Role-based: Expert personas
4. **Output Parsers**: String, Pydantic, JSON
5. **LCEL Chains**: Composing multiple steps
6. **Best Practices**: Clear instructions, parameter tuning
7. **Real-world Examples**: Summarization, extraction, Q&A

### Key Takeaways:
- Choose the right prompting technique based on your task complexity
- Use few-shot prompting for better accuracy without fine-tuning
- Chain-of-Thought improves reasoning on complex problems
- Structure outputs with parsers for downstream processing
- Always iterate and refine your prompts based on results